  Author: Ankit Kariryaa, University of Bremen
  
  Modified by Xuehui Pi, Qiuqi Luo and Beihui Hu



In [ ]:
from tensorflow.keras.models import load_model
import numpy as np               # numerical array manipulation
import pandas as pd
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import time
from functools import reduce
from PIL import Image
import rasterio                  # I/O raster data (netcdf, height, geotiff, ...)
import rasterio.warp             # Reproject raster samples
import rasterio.mask
from core.losses import accuracy, dice_loss, IoU, recall,F1_score, precision
from core.optimizers import adaDelta
from core.frame_info import FrameInfo
from core.dataset_generator import DataGenerator
from core.visualize import display_images

from tensorflow.keras import mixed_precision 
mixed_precision.set_global_policy('mixed_float16')

import warnings                  # ignore annoying warnings
warnings.filterwarnings("ignore")
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

%reload_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'

import tensorflow as tf
print(tf.__version__)

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto(
    #device_count={"CPU": 64},
    allow_soft_placement=True, 
    log_device_placement=False)
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
# Initialize the data related variables used in the notebook 

# For reading the GSW and annotated images generated in the step - 1

base_dir = r''
dataset_dir=os.path.join(base_dir,'dataset')
type_num = 5
image_type = '.tif'
ann_type = '.png'
annotation_fn = 'annotation_type'
image_fn = 'image_type'
# For testing, images are divided into sequential patches 
patch_generation_stratergy = 'sequential'
patch_size = (576,576,6) ## Height * Width * (Input or Output) channels：[GSW, ANNOTATION]
BATCH_SIZE = 16 # Model is evaluated in batches; See https://keras.io/models/model/

# # When stratergy == sequential
step_size = (576,576)

input_shape = (576,576,5)
input_image_channel = [0,1,2,3,4]
input_label_channel = [5]

OPTIMIZER_NAME = 'adaDelta'
OPTIMIZER = adaDelta 
# OPTIMIZER=tf.train.experimental.enable_mixed_precision_graph_rewrite(OPTIMIZER)
OPTIMIZER =  mixed_precision.LossScaleOptimizer(OPTIMIZER)

In [ ]:
LOSS = dice_loss
LOSS_NAME = 'dice_loss'
modelToEvaluate =os.path.join(base_dir,r'saved_models\lakes_20240130-2243_AdaDelta_dice_loss_012345_576.h5')
print(modelToEvaluate)

In [ ]:
#File path for final report 
timestr = time.strftime("%Y%m%d-%H%M")
chf = input_image_channel + input_label_channel
chs = reduce(lambda a,b: a+str(b),   chf, '')
evaluation_report_path = model_path =  os.path.join(base_dir, 'evaluationreport') 
if not os.path.exists(evaluation_report_path):
    os.makedirs(evaluation_report_path)
evaluation_report_filename = os.path.join(evaluation_report_path,'evaluation_per_pixel{}_{}.csv'.format(timestr,chs))
print(evaluation_report_filename) 

In [ ]:
def readImgs(path_to_write, fn):
    image = rasterio.open(os.path.join(path_to_write, fn))
    read_image = image.read()
    comb_img = np.transpose(read_image, axes=(1,2,0))
    annotation_im = Image.open(os.path.join(path_to_write, fn.replace(image_fn,annotation_fn).replace(image_type,ann_type)))
    annotation = np.array(annotation_im)
    f = FrameInfo(comb_img, annotation)
    return f

def readFrames(dataType):
    frames=[]
    print(dataType)
    ds_dir=os.path.join(dataset_dir,'{}'.format(dataType))
    all_files = os.listdir(ds_dir)
    all_files_image = [fn for fn in all_files if fn.startswith(image_fn) and fn.endswith(image_type)]
    for j, fn in enumerate(all_files_image):
        f = readImgs(ds_dir,fn)
        frames.append(f)
    return frames

In [ ]:
frames=readFrames('test')
test_patches = DataGenerator(input_image_channel,patch_size, frames, input_label_channel, augmenter = None).all_sequential_patches(step_size)
print('test patchs number:',len(test_patches[0]))

In [ ]:
#Display the some of the test images
numberOfImagesToDisplay = 5

train_images, real_label = test_patches[0][:numberOfImagesToDisplay], test_patches[1][:numberOfImagesToDisplay]
display_images(np.concatenate((train_images,real_label), axis = -1))

In [ ]:
#Evaluate model 

def evaluate_model(model_path, evaluation_report_filename):
    print(model_path, evaluation_report_filename)
    model = load_model(modelToEvaluate, custom_objects={'dice_loss':dice_loss, 'accuracy':accuracy , 'recall':recall, 'precision':precision,'IoU':IoU,'F1_score':F1_score}, compile=False)
    model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=[dice_loss, accuracy,recall,precision,F1_score,IoU])
    
    print('Evaluating model now!')
    ev = model.evaluate(test_patches[0], test_patches[1],  verbose=1, use_multiprocessing=False)
    report  = dict(zip(model.metrics_names, ev))
    report['model_path'] =  model_path   
    report['test_frame_dir']= base_dir   
    report['total_patch_count']= len(test_patches[0])  
    return report

report = evaluate_model(modelToEvaluate, evaluation_report_filename)

In [ ]:
# Generate the final report
print(report)

tdf = pd.DataFrame(report, index=[0])  
print(tdf.columns)
col_beginning = ['model_path','test_frame_dir', 'total_patch_count', 'accuracy', 'recall','precision','IoU','F1_score']

col_rest = [x for x in tdf.columns.tolist() if x not in col_beginning]
cols = col_beginning + col_rest
tdf = tdf[cols]
tdf.to_csv(evaluation_report_filename)

In [ ]:
model = load_model(modelToEvaluate, custom_objects={'dice_loss':dice_loss, 'accuracy':accuracy , 'recall':recall, 'precision':precision,'IoU':IoU,'F1_score':F1_score}, compile=False)
model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=[dice_loss, accuracy,recall,precision,F1_score,IoU])
columns=['model_path','test_frame_dir', 'total_patch_count', 'accuracy', 'recall','precision','IoU','loss','dice_loss','F1_score']
tdf=pd.DataFrame(columns=columns)

ds_dir=os.path.join(dataset_dir,'test')
all_files = os.listdir(ds_dir)
# print(all_files)

In [ ]:
tdf=pd.DataFrame(columns=columns)
for i in range(type_num):
    patches = []
    frames=[]
    files_image = [fn for fn in all_files if fn.startswith(image_fn+str(i)) and  fn.endswith(image_type)]#image.png
    print('type{} image number:{}'.format(i,len(files_image)))
    for j, fn in enumerate(files_image):
        f= readImgs(ds_dir,fn)
        frames.append(f)
    for frame in frames:
        ps= frame.sequential_patches(patch_size, step_size)
        patches.extend(ps)
    data = np.array(patches)
    del frames,patches
    ev = model.evaluate(data[...,:5],data[...,[5]],verbose=1, use_multiprocessing=False)
    report  = dict(zip(model.metrics_names, ev))
    report['model_path'] =  modelToEvaluate   
    report['test_frame_dir']= ds_dir   
    report['total_patch_count']= len(data)  

    new_row = pd.DataFrame([report], index=[0])
    tdf = pd.concat([tdf, new_row], ignore_index=True)

evaluation_report_filename = os.path.join(evaluation_report_path,'evaluation_per_pixel{}_{}.csv'.format(timestr,chs))
tdf.to_csv(evaluation_report_filename)

In [ ]:
titles=['ndwi','rgb','swir','annotation','prediction']
test_images = data[i*8:i*8+8,...,:5]
real_label = data[i*8:i*8+8,...,[5]]
prediction = model.predict(test_images, steps=1)
prediction[prediction>0.5]=1
prediction[prediction<=0.5]=0
display_images(np.concatenate((test_images, real_label, prediction), axis = -1),titles=titles)
i=i+1